# Lesson 4.4: Custom Agents and Agent Executor

---

In the previous lesson, we explored common Agent types like Zero-shot ReAct Agent and Conversational Agent. While these built-in Agents are powerful, sometimes you need an Agent with unique reasoning logic or behavior that doesn't fit the available patterns. This lesson will guide you on how to build **Custom Agents** and how to control their behavior through the **Agent Executor**.

## 1. Building Agents with Custom Logic (Custom Agents)

### 1.1. When are Custom Agents Needed?

You should consider building a Custom Agent when:

* **Complex Reasoning Logic:** Built-in Agent strategies (like ReAct) might not be sufficient for very complex or non-standard reasoning scenarios. You want precise control over how the LLM generates "Thought" and "Action."

* **Specific Response Format:** You need the Agent to produce actions or responses in a particular format not supported by default Agents.

* **Deep Integration with Your System:** You want the Agent to have a specific way of interacting with other components in your architecture that isn't just a standalone "Tool."

* **Performance Optimization:** You want to fine-tune the reasoning-action loop to achieve better performance for your specific use case.



### 1.2. Approaches to Building Custom Agents

Building a Custom Agent typically involves redefining how the LLM reasons and formats its output so that the Agent Executor can understand and execute it. This is often done by customizing the Prompt and potentially the Output Parser.

* **Prompt Customization:** This is the most common approach. You will create a very detailed `ChatPromptTemplate` that instructs the LLM on how to think, when to use a tool, and how to format its actions.

* **Output Parser Customization:** If your LLM outputs actions in a non-standard format, you will need a custom `OutputParser` to parse the LLM's output into `AgentAction` or `AgentFinish` objects that the Agent Executor can understand.


---

## 2. Controlling Agent Behavior via AgentExecutor

The **Agent Executor** is the component responsible for orchestrating the Agent's reasoning-action loop. It provides several important parameters that allow you to control the Agent's behavior.

### 2.1. Key Parameters of `AgentExecutor`

* **`agent`**: The Agent object that the Executor will run (e.g., an Agent created by `create_react_agent`).
* **`tools`**: The list of tools that the Agent can use.
* **`verbose`**: (Boolean, default `False`) If `True`, the Agent Executor will print out the Agent's thoughts, actions, and observations, which is very useful for debugging and understanding the flow.
* **`handle_parsing_errors`**: (Boolean or string, default `False`) Controls how parsing errors in the LLM's output are handled. If `True`, errors will be passed back to the LLM for it to attempt self-correction.
* **`max_iterations`**: (Integer, default `15`) The maximum number of iterations the Agent Executor will perform before stopping. This helps prevent infinite loops.
* **`max_execution_time`**: (Float) The maximum time (in seconds) the Agent Executor will run before stopping.
* **`return_intermediate_steps`**: (Boolean, default `False`) If `True`, the `invoke` output will include a list of the intermediate steps (Thought, Action, Observation) the Agent took.




---

## 3. Handling Errors and Infinite Loops in Agents

Agents, due to their dynamic nature, can encounter issues such as parsing errors or infinite loops.

### 3.1. Handling Parsing Errors

The LLM can sometimes generate output that doesn't conform to the format expected by the Agent Executor (e.g., incorrect syntax for tool calls).

* **Causes:** Prompt is not clear enough, LLM hallucination, or LLM `temperature` is too high.
* **Solutions:**
    * **Improve Prompt:** Make the prompt clearer, provide examples of the desired action format.
    * **`handle_parsing_errors=True`:** Allows the Agent Executor to pass the parsing error back to the LLM as an Observation. The LLM can then attempt to self-correct in the next turn.
    * **Lower `temperature`:** Helps the LLM generate more consistent output.

### 3.2. Handling Infinite Loops

An Agent can fall into an infinite loop if it continuously takes ineffective actions or fails to achieve its goal.

* **Causes:**
    * The LLM cannot reason correctly to break out of a situation.
    * Tools return unhelpful or misleading results.
    * Lack of appropriate information or tools to solve the problem.
* **Solutions:**
    * **`max_iterations` and `max_execution_time`:** These are the most important safety measures. They ensure the Agent will stop after a certain number of steps or time, preventing wasted resources.
    * **Improve Prompt:** Instruct the LLM on how to recognize when to stop or when to state that it cannot solve the problem.
    * **Improve Tools:** Ensure tools return clear and useful Observations.
    * **Add an "escape hatch" tool:** For example, a tool named `give_up` that the Agent can call when it cannot find a solution.


---

## 4. Practical Example: Building an Agent Capable of Interacting with an External API (Weather API)

We will build a simple Agent capable of looking up weather information by interacting with a mock weather API via a Custom Tool.

**Preparation:**
* Ensure you have `langchain-openai` installed.
* Set the `OPENAI_API_KEY` environment variable.

In [ ]:
# Install libraries if not already installed
# pip install langchain-openai openai pydantic

import os
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.tools import BaseTool, tool
from pydantic import BaseModel, Field
from typing import Type

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# 1. Initialize LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# 2. Create a Custom Tool to interact with a mock weather API
# We will use the @tool decorator for simplicity

@tool
def get_weather_data(location: str) -> str:
    """
    Get current weather information for a specific location.
    Input is the city or location name (e.g., "Hanoi", "Da Nang", "Tokyo").
    Returns a string describing the weather.
    """
    location_lower = location.lower()
    if "hanoi" in location_lower:
        return "Hanoi Weather: 28°C, sunny, 70% humidity, light wind."
    elif "da nang" in location_lower:
        return "Da Nang Weather: 30°C, clear sky, 65% humidity, no rain."
    elif "london" in location_lower:
        return "London Weather: 15°C, cloudy, light drizzle, strong wind."
    elif "tokyo" in location_lower:
        return "Tokyo Weather: 25°C, sunny, 60% humidity."
    else:
        return f"Weather information not found for location: {location}."

# Gather all Tools the Agent can use
tools = [get_weather_data]

# 3. Define Prompt for the Agent
# This prompt will instruct the LLM on how to think and use the get_weather_data tool.
# MessagesPlaceholder(variable_name="agent_scratchpad") is crucial for the Agent to record its Thoughts, Actions, and Observations.
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful weather assistant. You have access to the 'get_weather_data' tool to look up weather. Use it to answer weather-related questions."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# 4. Create Agent
agent = create_react_agent(llm, tools, prompt)

# 5. Create Agent Executor
# Configure max_iterations to prevent infinite loops
# Configure handle_parsing_errors so the Agent can attempt self-correction
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True, # To see the thought process
    max_iterations=5, # Limit the number of steps to prevent infinite loops
    handle_parsing_errors=True # Allow the Agent to attempt self-correction for parsing errors
)

# --- Execute the Agent ---
print("--- Starting Agent Executor with Custom Weather Tool ---")
chat_history = []

# Question 1: Requires weather lookup
query_1 = "What's the weather like today in Da Nang?"
print(f"\nUser: {query_1}")
response_1 = agent_executor.invoke({"input": query_1, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=query_1), AIMessage(content=response_1["output"])])
print(f"Agent: {response_1['output']}")

# Question 2: Requires weather lookup for a location with no data
query_2 = "What's the weather like in Sydney right now?"
print(f"\nUser: {query_2}")
response_2 = agent_executor.invoke({"input": query_2, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=query_2), AIMessage(content=response_2["output"])])
print(f"Agent: {response_2['output']}")

# Question 3: Question outside the Agent's capability (no suitable tool)
query_3 = "Calculate the sum of 123 and 456."
print(f"\nUser: {query_3}")
response_3 = agent_executor.invoke({"input": query_3, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=query_3), AIMessage(content=response_3["output"])])
print(f"Agent: {response_3['output']}")

print("--- Agent Executor Ended ---")

**Explanation:**
* We created a simple Python function `get_weather_data` and turned it into a Tool using the `@tool` decorator. This Tool simulates calling a weather API.
* The `AgentExecutor` is configured with `max_iterations=5` to limit the number of steps the Agent can take, and `handle_parsing_errors=True` so it can attempt to self-correct if the LLM produces malformed output.
* When you ask a weather-related question, the Agent will use the `get_weather_data` Tool to look up and answer. If the location is not in the mock data, the Tool will return an error message, and the Agent will relay that message.
* For the calculation question, since the Agent does not have a `Calculator` tool (only `get_weather_data`), it will attempt to answer using the LLM's own knowledge or state that it cannot perform the task.


---

## Lesson Summary

This lesson provided in-depth knowledge about **Custom Agents** and how to control them through the **Agent Executor**. You understood when to build an Agent with custom logic and how `AgentExecutor` parameters like `max_iterations` and `handle_parsing_errors` help manage Agent behavior. We also discussed how to **handle parsing errors** and **prevent infinite loops**. Finally, you practiced **building an Agent capable of interacting with an external API** (a mock weather API) by creating a Custom Tool and integrating it into the Agent Executor. Mastering these techniques is key to designing and deploying powerful, reliable, and problem-solving Agents in the real world.